## 动态时间序列规整

我们在登山时候会驻足观赏连绵的山脉，这段像什么，那段像什么，这时候也可能是比较两段山脉的像不像，这里我们就会发现每个人看到可能都不一样，因为观察的起点和终点是不同的，也可能大小是不同的，都是靠想象了。  
类似，我们平时见到的股票波动数据，心电图数据，语音数据等都是一种时间序列，而我们会转为比较两条曲线的相似度来比较两段语音是否相似，这时候的两段时间序列就可能不一样长短，因为每个人的语速是不同的，也可能有同音词，也可能会不同的语调等等。这种情况下，利用传统的欧几里得距离是无法求得有效的两个时间序列之间的相似性的（即距离）。  

动态时间规整 Dynamic Time Warping（DTW）就是一种衡量两个时间序列之间的相似度的方法，又是个日本学者提出的，名叫Itakura，DTW通过把时间序列进行延伸和缩短，来计算两个时间序列性之间的相似性：（比如识别两段语音是否表示同一个单词）  

![image.png](images/dtw3.jpg)

   将两个时间序列x和y的按时间轴互相垂直放置，现在关键是找到起点和终点，如果两个序列长度相等，那就直接计算两个的距离就好了。
但如果不相等我们就需要对齐。最简单的对齐方式就是线性缩放了。把短的序列线性放大到和长序列一样的长度再比较，或者把长的线性缩短到和短序列一样的长度再比较。但是这样的计算没有考虑到语音中各个段在不同情况下的持续时间会产生或长或短的变化，因此识别效果不可能最佳。因此更多的是采用动态规划（dynamic programming）的方法。  
   为了对齐这两个序列，我们需要构造一个n x m的矩阵网格，矩阵元素(i, j)表示qi和cj两个点的距离d(qi, cj)（也就是序列Q的每一个点和C的每一个点之间的相似度，距离越小则相似度越高。这里先不管顺序），一般采用欧式距离，d(qi, cj)= (qi-cj)2（也可以理解为失真度）。每一个矩阵元素(i, j)表示点qi和cj的对齐。DP算法可以归结为寻找一条通过此网格中若干格点的路径，路径通过的格点即为两个序列进行计算的对齐的点。


###  DTW计算方法：

令要计算相似度的两个时间序列为X和Y，长度分别为|X|和|Y|。

归整路径(Warp Path)

归整路径的形式为W=w1,w2,...,wK，其中Max(|X|,|Y|)<=K<=|X|+|Y|。

wk的形式为(i,j)，其中i表示的是X中的i坐标，j表示的是Y中的j坐标。

归整路径W必须从w1=(1,1)开始，到wK=(|X|,|Y|)结尾，以保证X和Y中的每个坐标都在W中出现。

另外，W中w(i,j)的i和j必须是单调增加的，以保证图1中的虚线不会相交，所谓单调增加是指：

$w_k = (i,j),w_{k+1}=(i^t,j^t)$ 
$ i<=i^t<=i+1 ,  j<=j^t<=j+1 $

最后要得到的归整路径是距离最短的一个归整路径：

D(i,j)=Dist(i,j)+min[D(i-1,j),D(i,j-1),D(i-1,j-1)]

最后求得的归整路径距离为D(|X|,|Y|)，使用动态规划来进行求解

简单讲DTW的思想是把两个时间序列进行延伸和缩短，来得到两个时间序列性距离最短也就是最相似的那一个warping，这个最短的距离也就是这两个时间序列的最后的距离度量。在这里，我们要做的就是选择一个路径，使得最后得到的总的距离最小。

https://blog.csdn.net/fewjioqpfjeiowph/article/details/83743573

https://blog.csdn.net/u011661040/article/details/16940613?utm_medium=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-8.nonecase&depth_1-utm_source=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-8.nonecase

https://blog.csdn.net/mystery_guest/article/details/82588286

https://blog.csdn.net/qq_39516859/article/details/81705010 

https://www.cnblogs.com/kemaswill/archive/2013/04/18/3029078.html

In [18]:
from math import *
import matplotlib.pyplot as plt
import numpy
 

 
def dtw_distance(ts_a, ts_b, d=lambda x,y: abs(x-y), mww=10000):
    """Computes dtw distance between two time series
    
    Args:
        ts_a: time series a
        ts_b: time series b
        d: distance function
        mww: max warping window, int, optional (default = infinity)
        
    Returns:
        dtw distance
    """
    
    # Create cost matrix via broadcasting with large int
    ts_a, ts_b = np.array(ts_a), np.array(ts_b)
    M, N = len(ts_a), len(ts_b)
    cost = np.ones((M, N))

    # Initialize the first row and column
    cost[0, 0] = d(ts_a[0], ts_b[0])
    for i in range(1, M):
        cost[i, 0] = cost[i-1, 0] + d(ts_a[i], ts_b[0])

    for j in range(1, N):
        cost[0, j] = cost[0, j-1] + d(ts_a[0], ts_b[j])

    # Populate rest of cost matrix within window
    for i in range(1, M):
        for j in range(max(1, i - mww), min(N, i + mww)):
            choices = cost[i-1, j-1], cost[i, j-1], cost[i-1, j]
            cost[i, j] = min(choices) + d(ts_a[i], ts_b[j])

    # Return DTW distance given window 
    return cost[-1, -1]

 
def display(s1, s2) :
 
    val, path = dtw(s1, s2, dist_for_float)
    
    w = len(s1)
    h = len(s2)
    
    mat = [[1] * w for i in range(h)]
    for node in path :
        x, y = node
        mat[y][x] = 0
 
    mat = numpy.array(mat)
    
    plt.subplot(2, 2, 2)
    c = plt.pcolor(mat, edgecolors='k', linewidths=4)
    plt.title('Dynamic Time Warping (%f)' % val)
 
    plt.subplot(2, 2, 1)
    plt.plot(s2, range(len(s2)), 'g')
    
    plt.subplot(2, 2, 4)
    plt.plot(range(len(s1)), s1, 'r')
        
 
    plt.show() 

In [21]:
#快速时间序列规整，o（n）
import numpy as np
import pandas as pd
import os
from  preutils import timeseries_processor
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
pd.set_option('display.width', 1000)
homePath = os.path.dirname(os.path.abspath('__file__'))
# Windows下的存储路径与Linux并不相同
#data//20200411/platform/db_oracle_11g.csv

if os.name == "nt":
    dataPath = "%s\\data\20200411\platform\dcos_docker.csv" % homePath
else:
    dataPath = "%s/data/20200411/platform/dcos_docker.csv" % homePath

itemid = 999999996381403
cmdbid = 'docker_003'
indexname = 'container_cpu_used'
bomcid = 'ZJ-004-059'

docker=pd.read_csv(dataPath)
#docker.head()
dockergrouped = docker.groupby(['itemid','name','bomc_id'])
#for item in dockergrouped.itemid:
alarmtime = [1586534700000]
dtdata  = docker[  (docker['itemid']==itemid) & (docker['cmdb_id']==cmdbid) & (docker['name']==indexname)][['timestamp','value']]
#timeseries = dtdata['value'].values
#dockertime1 = docker.loc[(docker['timestamp'] <= alarmtime)&(docker['timestamp']>alarmtime- 300000)]
#timeseries_set =  dockertime1['value'].values
#print(dtdata.shape)

dts = timeseries_processor.get_tsdata(dataPath,itemid,bomcid,indexname )
print(dts.head())
print(dtdata.head())
#dts.plot()
y = np.random.randint(0,100,[356,]) 

distance = dtw_distance(dts, y, d=euclidean)
print(distance)
distance, path = fastdtw(dts, y, dist=euclidean)
print(distance)
distance,path = fastdtw(dts,dts,dist=euclidean)
print(distance)


2020-04-11 00:00:28    1.0
2020-04-11 00:01:28    2.0
2020-04-11 00:02:28    2.0
2020-04-11 00:03:02    1.0
2020-04-11 00:04:02    2.0
dtype: float64
         timestamp  value
20   1586534428000    1.0
350  1586534727000   12.0
351  1586534642000    2.0
352  1586534582000    1.0
353  1586534548000    2.0
9693.0
13884.0
0.0
